# Tune parameters for NN with Keras Tuner

In [1]:
# Download Keras Tuner
!git clone https://github.com/keras-team/keras-tuner.git

Cloning into 'keras-tuner'...
remote: Enumerating objects: 8999, done.
remote: Counting objects: 100% (8999/8999), done.
remote: Compressing objects: 100% (2510/2510), done.
remote: Total 8999 (delta 6448), reused 8845 (delta 6363), pack-reused 0
Receiving objects: 100% (8999/8999), 2.06 MiB | 5.23 MiB/s, done.
Resolving deltas: 100% (6448/6448), done.


In [2]:
# Install Keras Tuner
%cd keras-tuner
!pip install .

/content/keras-tuner
Processing /content/keras-tuner
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 9.5 MB/s eta 0:00:00
  Created wheel for keras-tuner: filename=keras_tuner-1.4.0.dev0-py3-none-any.whl size=182771 sha256=aa7b4cb13e0e6f4385e496342e09d13589b544c41f40f461c634de6413b0262a
  Stored in directory: /root/.cache/pip/wheels/6a/f2/f6/4d216b7ba0b7c0374eb8c129c16da679bd15329b761cbad121
Successfully built keras-tuner


In [3]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from google.colab import files
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image
from kerastuner import RandomSearch, Hyperband, BayesianOptimization

Using TensorFlow backend


<ipython-input-3-bee5f6b8c352>:11: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch, Hyperband, BayesianOptimization


In [4]:
# Preprocess data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60_000, 784)
x_test = x_test.reshape(10_000, 784)

# Normalize data
x_train = x_train / 255
x_test = x_test / 255

# One-hot encoding
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
# Create model
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_input',
                                   min_value=128,
                                   max_value=1024,
                                   step=32),
                    input_dim=784,
                    activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [6]:
# Create Tuner
tuner = RandomSearch(
    hypermodel=build_model,
    objective='val_accuracy',  # metrics to optimize
    max_trials=10,  # max quantity of model's launches with different values of hyperparemeters
    directory='test_directory'  # folder where model will be saved
)
tuner

# [RandomSearch Tuner](https://keras.io/api/keras_tuner/tuners/random/)

In [7]:
# Search space
tuner.search_space_summary()

Search space summary
Default search space size: 1
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}


In [8]:
# Choose hyperparameters
tuner.search(
    x_train,
    y_train,
    batch_size=256,  # mini-sample size
    epochs=3,
    validation_split=0.2,
    verbose=1  # regime of output info
)

Trial 10 Complete [00h 00m 05s]
val_accuracy: 0.7867500185966492

Best val_accuracy So Far: 0.7889166474342346
Total elapsed time: 00h 00m 59s


# Get best model

In [9]:
tuner.results_summary()

Results summary
Results in test_directory/untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 04 summary
Hyperparameters:
units_input: 896
Score: 0.7889166474342346

Trial 09 summary
Hyperparameters:
units_input: 480
Score: 0.7867500185966492

Trial 02 summary
Hyperparameters:
units_input: 992
Score: 0.7861666679382324

Trial 01 summary
Hyperparameters:
units_input: 352
Score: 0.7825833559036255

Trial 03 summary
Hyperparameters:
units_input: 544
Score: 0.7819166779518127

Trial 00 summary
Hyperparameters:
units_input: 416
Score: 0.781499981880188

Trial 06 summary
Hyperparameters:
units_input: 512
Score: 0.7797499895095825

Trial 08 summary
Hyperparameters:
units_input: 576
Score: 0.7794166803359985

Trial 05 summary
Hyperparameters:
units_input: 288
Score: 0.7765833139419556

Trial 07 summary
Hyperparameters:
units_input: 320
Score: 0.7764166593551636


In [10]:
# Three best models
models = tuner.get_best_models(num_models=3)

In [11]:
# Evaluate quality of models on test data
for model in models:
  model.summary()
  model.evaluate(x_test, y_test)
  print()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 896)               703360    
                                                                 
 dense_1 (Dense)             (None, 10)                8970      
                                                                 
Total params: 712330 (2.72 MB)
Trainable params: 712330 (2.72 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
313/313 [==============================] - 1s 3ms/step - loss: 0.6875 - accuracy: 0.7800

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 480)               376800    
                                                                 
 dense_1 (Dense)             (None, 10)   

# Let's add one hidden layer to our model!

In [12]:
# Add hidden layer in our NN architecture
def build_new_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_input',
                                   min_value=512,
                                   max_value=1024,
                                   step=32),
                    input_dim=784,
                    activation='relu'))
    # Add hidden layer
    model.add(Dense(units=hp.Int('units_hidden',
                                 min_value=128,
                                 max_value=600,
                                 step=32),
                      activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [13]:
# Use Bayesian optimizator
tuner = BayesianOptimization(
    build_new_model,
    objective='val_accuracy',
    max_trials=10,
    directory='test_directory',
    overwrite=True
)

In [14]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
units_hidden (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 600, 'step': 32, 'sampling': 'linear'}


In [15]:
tuner.search(
    x_train,
    y_train,
    batch_size=256,  # mini-sample size
    epochs=3,
    validation_split=0.2,
    verbose=1  # regime of output info
)

Trial 10 Complete [00h 00m 06s]
val_accuracy: 0.8004166483879089

Best val_accuracy So Far: 0.8010833263397217
Total elapsed time: 00h 01m 03s


In [16]:
tuner.results_summary()

Results summary
Results in test_directory/untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 06 summary
Hyperparameters:
units_input: 992
units_hidden: 288
Score: 0.8010833263397217

Trial 03 summary
Hyperparameters:
units_input: 800
units_hidden: 352
Score: 0.8004999756813049

Trial 09 summary
Hyperparameters:
units_input: 960
units_hidden: 288
Score: 0.8004166483879089

Trial 04 summary
Hyperparameters:
units_input: 928
units_hidden: 544
Score: 0.7983333468437195

Trial 01 summary
Hyperparameters:
units_input: 800
units_hidden: 512
Score: 0.7980833053588867

Trial 00 summary
Hyperparameters:
units_input: 704
units_hidden: 320
Score: 0.7960000038146973

Trial 02 summary
Hyperparameters:
units_input: 576
units_hidden: 256
Score: 0.7944166660308838

Trial 07 summary
Hyperparameters:
units_input: 864
units_hidden: 128
Score: 0.7925000190734863

Trial 08 summary
Hyperparameters:
units_input: 1024
units_hidden: 416
Score: 0.7885000109672546

Tria

In [17]:
new_models = tuner.get_best_models(num_models=3)

In [18]:
for model in models:
    model.summary()
    model.evaluate(x_test, y_test)
    print()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 896)               703360    
                                                                 
 dense_1 (Dense)             (None, 10)                8970      
                                                                 
Total params: 712330 (2.72 MB)
Trainable params: 712330 (2.72 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
313/313 [==============================] - 1s 3ms/step - loss: 0.6875 - accuracy: 0.7800

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 480)               376800    
                                                                 
 dense_1 (Dense)             (None, 10)   

In [19]:
# Let's find best activation function and optimizer using hyperparameters
def build_hp_model(hp):
    model = Sequential()
    # Activation choice
    activation_choise = hp.Choice('activation',
                                  values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])
    model.add(Dense(units=hp.Int('units_input',
                                   min_value=512,
                                   max_value=1024,
                                   step=32),
                    input_dim=784,
                    activation=activation_choise))
    # Add hidden layer
    model.add(Dense(units=hp.Int('units_hidden',
                                 min_value=128,
                                 max_value=600,
                                 step=32),
                      activation=activation_choise))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop', 'SGD']),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# [Getting started with KerasTuner](https://keras.io/guides/keras_tuner/getting_started/)

In [20]:
new_tuner = BayesianOptimization(
    build_hp_model,
    objective='val_accuracy',
    max_trials=5,
    directory='test_directory',
    overwrite=True
)

In [21]:
new_tuner.search_space_summary()

Search space summary
Default search space size: 4
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh', 'elu', 'selu'], 'ordered': False}
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
units_hidden (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 600, 'step': 32, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop', 'SGD'], 'ordered': False}


In [22]:
new_tuner.search(
    x_train,
    y_train,
    batch_size=256,  # mini-sample size
    epochs=3,
    validation_split=0.2,
    verbose=1  # regime of output info
)

Trial 5 Complete [00h 00m 07s]
val_accuracy: 0.8319166898727417

Best val_accuracy So Far: 0.8709166646003723
Total elapsed time: 00h 00m 28s


In [23]:
new_tuner.results_summary()

Results summary
Results in test_directory/untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
activation: selu
units_input: 672
units_hidden: 256
optimizer: adam
Score: 0.8709166646003723

Trial 3 summary
Hyperparameters:
activation: elu
units_input: 576
units_hidden: 160
optimizer: adam
Score: 0.8616666793823242

Trial 2 summary
Hyperparameters:
activation: tanh
units_input: 544
units_hidden: 352
optimizer: rmsprop
Score: 0.8533333539962769

Trial 4 summary
Hyperparameters:
activation: selu
units_input: 992
units_hidden: 384
optimizer: rmsprop
Score: 0.8319166898727417

Trial 1 summary
Hyperparameters:
activation: selu
units_input: 576
units_hidden: 512
optimizer: SGD
Score: 0.8202499747276306


In [24]:
best_model = new_tuner.get_best_models(1)
best_model[0].summary()
best_model[0].evaluate(x_test, y_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 672)               527520    
                                                                 
 dense_1 (Dense)             (None, 256)               172288    
                                                                 
 dense_2 (Dense)             (None, 10)                2570      
                                                                 
Total params: 702378 (2.68 MB)
Trainable params: 702378 (2.68 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
313/313 [==============================] - 1s 3ms/step - loss: 0.3997 - accuracy: 0.8564


[0.3997249901294708, 0.8564000129699707]

In [25]:
# Add layers in our model
def big_model(hp):
    model = Sequential()
    # Activation choice
    activation_choise = hp.Choice('activation',
                                  values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])
    model.add(Dense(units=hp.Int('units_input',
                                   min_value=512,
                                   max_value=1024,
                                   step=32),
                    input_dim=784,
                    activation=activation_choise))
    # Add hidden layers
    for layer in range(hp.Int('num_layers', 2, 5)):
        model.add(Dense(units=hp.Int('units_hidden',
                                    min_value=128,
                                    max_value=600,
                                    step=32),
                          activation=activation_choise))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop', 'SGD']),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Last model helps to even set quantity of hidden layers